# SmolVLA 模型训练 (MuJoCo 数据)

这个 Notebook 使用从 MuJoCo 环境采集的数据训练 SmolVLA 模型。

## ⚠️ 注意事项

- **需要 GPU**: 推荐 A100 或 RTX 3090+
- **训练时间**: 约 4-8 小时（取决于 GPU）
- **磁盘空间**: 确保有足够空间保存 checkpoints (约 10-20GB)
- **Sim2Real Gap**: MuJoCo 训练的模型不能直接用于真实机械臂

## 📈 预计训练时间

- A100 GPU: ~4 小时
- RTX 3090: ~6-8 小时  
- RTX 3080: ~8-10 小时
- CPU: 不推荐（太慢）

---
## 📦 步骤 0: 导入库和环境检查

In [ ]:
import logging
from pathlib import Path

import torch

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 库导入成功！")

---
## 💻 步骤 1: 检查 GPU

In [ ]:
print("🔍 检查 GPU...\n")

if not torch.cuda.is_available():
    print("⚠️ 未检测到 CUDA GPU!")
    print("训练将非常慢。推荐使用:")
    print("  - Google Colab (免费 GPU)")
    print("  - AWS/GCP 云 GPU")
    print("  - 本地 NVIDIA GPU")
    print()
    
    # 询问是否继续
    response = input("是否继续使用 CPU 训练? (y/n): ")
    if response.lower() != 'y':
        raise RuntimeError("需要 GPU 进行训练")
    has_gpu = False
else:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ 检测到 GPU: {gpu_name}")
    print(f"✓ GPU 显存: {gpu_memory:.1f} GB")
    has_gpu = True

print()

---
## ⚙️ 步骤 2: 配置训练参数

In [ ]:
# ========== 数据集配置 ==========
DATASET_REPO_ID = "your_username/mujoco_so101_pickplace"  # 修改为您的数据集

# ========== 模型配置 ==========
POLICY_PATH = "lerobot/smolvla_base"  # 预训练模型路径

# ========== 训练参数 ==========
BATCH_SIZE = 64  # A100: 64, RTX 3090: 32-48, RTX 3080: 24-32
STEPS = 20000  # 训练步数
LEARNING_RATE = 1e-4  # 学习率
EVAL_FREQ = 1000  # 评估频率（每 N 步）
SAVE_FREQ = 2000  # 保存 checkpoint 频率（每 N 步）

# ========== 输出配置 ==========
OUTPUT_DIR = "outputs/mujoco_smolvla"  # 输出目录
JOB_NAME = "smolvla_mujoco_so101"  # 任务名称

# ========== W&B 配置 ==========
USE_WANDB = True  # 使用 Weights & Biases 记录训练
WANDB_PROJECT = "lerobot-mujoco"  # W&B 项目名称
WANDB_RUN_NAME = JOB_NAME  # W&B run 名称

# ========== 设备配置 ==========
DEVICE = "cuda" if has_gpu else "cpu"
NUM_WORKERS = 4  # 数据加载器的工作进程数

# 如果使用 CPU，减小 batch size
if DEVICE == "cpu":
    BATCH_SIZE = min(BATCH_SIZE, 8)

# 创建输出目录
output_dir = Path(OUTPUT_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

print("✅ 训练配置:")
print(f"  数据集: {DATASET_REPO_ID}")
print(f"  模型: {POLICY_PATH}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  训练步数: {STEPS}")
print(f"  学习率: {LEARNING_RATE}")
print(f"  输出目录: {OUTPUT_DIR}")
print(f"  使用 W&B: {USE_WANDB}")
print(f"  设备: {DEVICE}")

---
## 🔐 步骤 3: 登录 Hugging Face 和 W&B

In [ ]:
# 登录 Hugging Face
from huggingface_hub import notebook_login

print("🔐 登录 Hugging Face...")
notebook_login()
print("✅ HF 登录成功！")

In [ ]:
# 登录 Weights & Biases (可选)
if USE_WANDB:
    import wandb
    print("🔐 登录 Weights & Biases...")
    wandb.login()
    print("✅ W&B 登录成功！")
else:
    print("⚠️ W&B 未启用，跳过登录")

---
## 📂 步骤 4: 加载数据集

In [ ]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset

print("📂 加载数据集...")

dataset = LeRobotDataset(
    repo_id=DATASET_REPO_ID,
    root=None,  # 自动下载到缓存
    download_videos=True
)

print(f"✅ 数据集加载成功")
print(f"  Episodes: {dataset.num_episodes}")
print(f"  Frames: {dataset.num_frames}")
print(f"  FPS: {dataset.fps}")
print(f"  特征: {list(dataset.features.keys())}")
print()

---
## 🧠 步骤 5: 加载预训练模型

In [ ]:
from lerobot.policies.smolvla import SmolVLAPolicy

print("🧠 加载预训练模型...")

device = torch.device(DEVICE)

# 加载预训练模型
policy = SmolVLAPolicy.from_pretrained(POLICY_PATH)
policy = policy.to(device)

param_count = sum(p.numel() for p in policy.parameters()) / 1e6

print(f"✅ 模型加载成功")
print(f"  参数量: {param_count:.1f}M")
print(f"  设备: {device}")
print()

---
## 📊 步骤 6: 训练说明

### 关于训练

由于完整的训练循环需要根据 LeRobot 的具体 API 实现，这里提供两种训练方式：

#### 方法 A: 使用 LeRobot 命令行工具 (推荐)

在终端运行以下命令：

```bash
python -m lerobot.scripts.train \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id=your_username/mujoco_so101_pickplace \
  --batch_size=64 \
  --steps=20000 \
  --lr=1e-4 \
  --output_dir=outputs/mujoco_smolvla \
  --job_name=smolvla_mujoco_so101 \
  --device=cuda \
  --wandb.enable=true \
  --wandb.project=lerobot-mujoco \
  --wandb.run_name=smolvla_mujoco_so101
```

#### 方法 B: 在 Notebook 中训练

运行下一个单元格生成训练命令，然后在终端执行。

### 训练监控

如果启用了 W&B，可以通过以下方式监控训练：

```
https://wandb.ai/your_username/lerobot-mujoco
```

在 Dashboard 中可以查看：
- Loss 曲线
- Learning rate
- GPU 使用率
- 训练速度 (steps/sec)

---
## 🚀 步骤 7: 生成训练命令

In [ ]:
# 生成训练命令
cmd_parts = [
    "python", "-m", "lerobot.scripts.train",
    f"--policy.path={POLICY_PATH}",
    f"--dataset.repo_id={DATASET_REPO_ID}",
    f"--batch_size={BATCH_SIZE}",
    f"--steps={STEPS}",
    f"--lr={LEARNING_RATE}",
    f"--output_dir={OUTPUT_DIR}",
    f"--job_name={JOB_NAME}",
    f"--device={DEVICE}",
]

if USE_WANDB:
    cmd_parts.extend([
        f"--wandb.enable=true",
        f"--wandb.project={WANDB_PROJECT}",
        f"--wandb.run_name={WANDB_RUN_NAME}",
    ])

train_command = " ".join(cmd_parts)

print("=" * 70)
print("🚀 训练命令")
print("=" * 70)
print()
print(train_command)
print()
print("=" * 70)
print()
print("💡 请在终端运行上述命令")
print()
print("如果命令不适用，请参考 LeRobot 文档:")
print("https://huggingface.co/docs/lerobot")

---
## 📈 步骤 8: 训练进度监控（可选）

如果您在终端运行训练，可以使用以下方法监控进度：

In [ ]:
# 查看最新的 checkpoint
import os

checkpoint_dir = Path(OUTPUT_DIR)
if checkpoint_dir.exists():
    checkpoints = sorted(checkpoint_dir.glob("checkpoint-*"))
    
    if checkpoints:
        print(f"📦 找到 {len(checkpoints)} 个 checkpoint:\n")
        for cp in checkpoints:
            step = cp.name.split("-")[-1]
            size_mb = sum(f.stat().st_size for f in cp.rglob('*')) / (1024 * 1024)
            print(f"  - {cp.name} ({size_mb:.1f} MB)")
        
        print()
        print(f"✅ 最新 checkpoint: {checkpoints[-1].name}")
    else:
        print("⚠️ 还没有找到 checkpoint")
        print("训练可能刚开始或还未开始")
else:
    print("⚠️ 输出目录不存在")
    print(f"请确认训练已开始，输出目录: {OUTPUT_DIR}")

---
## 📊 步骤 9: 查看训练日志（可选）

In [ ]:
# 查看最新的训练日志
log_dir = Path(OUTPUT_DIR)
if log_dir.exists():
    log_files = list(log_dir.glob("*.log"))
    
    if log_files:
        latest_log = sorted(log_files, key=lambda x: x.stat().st_mtime)[-1]
        print(f"📄 最新日志文件: {latest_log}")
        print()
        print("最后 50 行：")
        print("=" * 70)
        
        with open(latest_log, 'r') as f:
            lines = f.readlines()
            for line in lines[-50:]:
                print(line.rstrip())
    else:
        print("⚠️ 没有找到日志文件")
else:
    print("⚠️ 输出目录不存在")

---
## ✅ 训练完成后的步骤

### 1. 加载训练好的模型

In [ ]:
# 训练完成后，加载最佳 checkpoint

checkpoint_dir = Path(OUTPUT_DIR)
checkpoints = sorted(checkpoint_dir.glob("checkpoint-*"))

if checkpoints:
    model_path = str(checkpoints[-1])  # 使用最新的 checkpoint
    print(f"📦 加载 checkpoint: {model_path}")
    
    trained_policy = SmolVLAPolicy.from_pretrained(model_path)
    trained_policy = trained_policy.to(device)
    trained_policy.eval()
    
    print("✅ 模型加载成功！")
    print(f"  设备: {device}")
    print(f"  参数量: {sum(p.numel() for p in trained_policy.parameters()) / 1e6:.1f}M")
else:
    print("⚠️ 没有找到训练好的模型")
    print("请先完成训练")

### 2. 下一步

训练完成后，您可以：

1. **评估模型**:
   - 打开 `03_evaluate_policy.ipynb`
   - 在 MuJoCo 环境中测试模型性能

2. **上传模型到 Hub**:
   - 分享您的模型给社区
   - 便于后续使用和部署

3. **优化和改进**:
   - 根据评估结果调整超参数
   - 采集更多高质量数据
   - 尝试不同的训练策略

---

**🎉 训练完成！继续下一步评估吧！**